In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv('amazon_cells_labelled.txt',delimiter='\t',names=['text','class'])

In [29]:
data_1 = pd.read_csv('imdb_labelled.txt',delimiter='\t',names=['text','class'])
data_2 = pd.read_csv('yelp_labelled.txt',delimiter='\t',names=['text','class'])

In [30]:
data_1.shape,data_2.shape

((748, 2), (1000, 2))

In [ ]:
#Concatenate data, data_1 and data_2

In [3]:
data.head()

,text,class
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [4]:
data.shape

(1000, 2)

In [5]:
data.groupby('class').count()

,text
class,
0,500
1,500


In [6]:
data.text = data.text.str.replace('\d','')
data.text = data.text.str.replace('[\.!,@#]+','')

In [7]:
my_sentences = data.text.values
my_sentences[:2]

array(['So there is no way for me to plug it in here in the US unless I go by a converter',
       'Good case Excellent value'], dtype=object)

In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [9]:
stem = PorterStemmer()

In [10]:
stop_words = stopwords.words('english')

In [11]:
tokenized = [word_tokenize(sent.lower()) for sent in my_sentences]

In [12]:
def remove_stop_words(text):
    return [stem.stem(word) for word in text if word not in stop_words]

In [13]:
tokenized = [remove_stop_words(text) for text in tokenized]

In [14]:
tokenized[:5]

[['way', 'plug', 'us', 'unless', 'go', 'convert'],
 ['good', 'case', 'excel', 'valu'],
 ['great', 'jawbon'],
 ['tie', 'charger', 'convers', 'last', 'minutesmajor', 'problem'],
 ['mic', 'great']]

In [15]:
target = data['class'].values

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3))

In [17]:
vectorizer.fit(data.text.values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 3), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [18]:
tokenized = [' '.join(sent) for sent in tokenized]

In [19]:
my_featured_text = vectorizer.transform(tokenized)

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(my_featured_text, target)

In [22]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(750, 14942)
(250, 14942)
(750,)
(250,)


In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
my_model = LogisticRegression()
my_model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
my_predictions = my_model.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [27]:
print(accuracy_score(y_test,my_predictions))

0.776


In [31]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [33]:
doc = nlp("Mr. Ramesh is going to Delhi for next two days")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Ramesh 4 10 PERSON
Delhi 23 28 GPE
next two days 33 46 DATE
